In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [ ]:
train = torchvision.datasets.CIFAR10(root = '. /data', train = True, download = True, transform = transforms.ToTensor())
train
test = torchvision.datasets.CIFAR10(root = '. /data', train = False, download = True, transform = transforms.ToTensor())
test

In [ ]:
labels = [item[1] for item in train]
num_classes = len(set(labels))
num_classes


In [ ]:
image, label = train[67]
print('Shape: ', image.shape, '\nLabel: ', label)



In [ ]:
plt.imshow(image.T, interpolation = 'bicubic')


In [ ]:
train_loader = torch.utils.data.DataLoader(train, batch_size=100,shuffle=True)
test_loader = torch.utils.data.DataLoader(test, batch_size=100,shuffle=True)




In [ ]:
#Neural Network

class ANN(nn.Module):
  def __init__(self,input_size, hidden_size1, hidden_size2, num_classes):
    super(ANN,self).__init__()
    self.fc1 = nn.Linear(input_size, hidden_size1, bias = True)
    self.relu1 = nn.ReLU()
    self.fc2 = nn.Linear(hidden_size1, hidden_size2, bias = True)
    self.relu2 = nn.ReLU()
    self.output = nn.Linear(hidden_size2, num_classes, bias = True)

  def forward(self,x):
    data = x.view(x.size(0), -1)
    data = self.fc1(data)
    data = self.relu1(data)
    data = self.fc2(data)
    data = self.relu2(data)
    data = self.output(data)
    return data

In [ ]:
x = torch.randn(2,3,4)
out = x.view(x.size(0),-1)
print(x.shape)
print(out.shape)

In [ ]:
input_size = 3 * 32* 32
hidden_size1 = 120
hidden_size2 = 100
num_classes = 10

model = ANN(input_size,hidden_size1, hidden_size2, num_classes)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.01)


In [ ]:
#Training

loss_list = []
accuracy_list = []

for epoch in range(100):
  epoch_loss = 0;
  for images,labels in train_loader:
    outputs = model(images)
    loss = criterion(outputs,labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    epoch_loss += loss.item()

  loss_list.append(epoch_loss/len(train_loader))

  with torch.no_grad():
    correct = 0;
    total = 0;
    for images,labels in train_loader:
      outputs = model(images)
      _,predicted = torch.max(outputs.data,1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()
      accuracy = correct/total
      accuracy_list.append(accuracy)

    print('Epoch [{}/100] Training Accuracy: {:.4f}'.format(epoch + 1, correct/total), 'Loss: {:.4f}'.format(loss_list[-1]))










In [ ]:
x = range(100)
plt.plot(x,loss_list)
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.show()



In [ ]:
#Testing

with torch.no_grad():
  correct = 0
  total = 0

  for images, labels in test_loader:
    pred_outputs = model(images)
    _,predicted = torch.max(pred_outputs.data,1)
    total += labels.size(0);
    correct += (predicted == labels).sum().item()

  print('Test Accuracy: {:.4f}'.format(correct/total))
  print('Test Accuracy Percentage: {:.4f}'.format(correct/total * 100))





In [ ]:
#Confusion Matrix

from sklearn.metrics import confusion_matrix

model.eval()
with torch.no_grad():
  correct = 0
  total = 0
  confusion_matrix = torch.zeros(10,10)
  for images,labels in test_loader:
    outputs = model(images)
    _,predicted = torch.max(outputs.data,1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
    for i,j in zip(predicted,labels):
      confusion_matrix[i][j] += 1
confusion_matrix = confusion_matrix.numpy().astype(int)
print('Confusion Matrix')
print(confusion_matrix)

In [ ]:
#Prediciton

import torch
import torchvision.transforms as transforms
from PIL import Image
import requests
from io import BytesIO

url = 'https://www.marineinsight.com/wp-content/uploads/2019/08/Cruise-ships-1.png.webp'
response = requests.get(url)
image = Image.open(BytesIO(response.content))

transform = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

plt.imshow(image)
plt.axis('off')
plt.show()

tensor = transform(image).unsqueeze(0)
output = model(tensor)

_,predicted = torch.max(output.data,1)
predicted_class = predicted.item()
dict = {0:"Airplane",1:"Automobile",2:"Bird",3:"Cat",4:"Deer",5:"Dog",6:"Frog",7:"Horse",8:"Ship",9:"Truck"}
print("This image is a", dict.get(predicted_class))